In [1]:
import pandas as pd

In [5]:
#data_spe = pd.read_csv("training_entities_subtask1.tsv", delimiter="\t")
data_spe = pd.read_csv("validation_entities_subtask1.tsv", delimiter="\t")
#data_spe = pd.read_csv("test_entities_subtask1.tsv", delimiter="\t")
data_spe

,filename,mark,label,off0,off1,span
0,32119083_ES,T1,HUMAN,287,294,familia
1,32119083_ES,T2,HUMAN,4827,4831,hijo
2,32119083_ES,T3,HUMAN,3948,3953,madre
3,32119083_ES,T4,HUMAN,4499,4504,madre
4,32119083_ES,T5,HUMAN,4567,4572,madre
...,...,...,...,...,...,...
7101,es-S1887-85712015000100005-1,T17,SPECIES,1798,1801,PNJ
7102,es-S1887-85712015000100005-1,T18,SPECIES,2574,2577,PNJ
7103,es-S1889-836X2015000400006-1,T1,SPECIES,61,64,VIH
7104,es-S1889-836X2015000400006-1,T2,HUMAN,1902,1910,paciente


In [6]:
#f_out = open("LivingNER_training.json", 'w', encoding="UTF-8")
f_out = open("LivingNER_validation.json", 'w', encoding="UTF-8")
#f_out = open("LivingNER_testing.json", 'w', encoding="UTF-8")
f_out.write('{"version": "0.1.0",\n"data": [')
#test_path = "text-files-tr"
test_path = "text-files-vl"
#test_path = "text-files-test"
test_files = os.listdir(test_path)
for f in test_files:
    with open(test_path+'/'+f, "r", encoding="UTF-8") as ftest:
        texto = ftest.read()
    fl = data_spe[data_spe['filename'] == f[:-4]].sort_values('off0', ascending=True)
    species = []
    species_lb = []
    for i in range(len(fl)):
        species.append(fl['span'].iloc[i])
        species_lb.append(fl['label'].iloc[i])
    for p, l in zip(species, species_lb):       
        if l == "SPECIES":
            tagged = [a+"___I#-#S#P#E#C#I#E" for a in p.split()]
        else:
            tagged = [a+"___I#-#H#U#M#A#N" for a in p.split()]

        txt_tag = ' '.join( tagged )
        #texto = re.sub(texto[start:end],txt_tag,texto)
        texto = texto.replace(p, txt_tag)
        texto = texto.replace('. ',' . ')
        texto = texto.replace('.\n',' . ')
        texto = texto.replace(', ',' , ')
        texto = texto.replace(': ',' : ')
        texto = texto.replace('! ',' ! ')
        texto = texto.replace('? ',' ? ')
        texto = texto.replace('; ',' ; ')
        texto = texto.replace(') ',' ) ')
        texto = texto.replace('" ',' \" ')
        texto = texto.replace(' (',' ( ')
        texto = texto.replace(' "',' \" ')
        texto = texto.replace('  ', ' ')
    texto = texto.split()
    ner_tags = []
    tokens = []
    for w in texto:
        #if w.endswith('B-DISEASE') or w.endswith('I-DISEASE'):
            #wp = w.split('___')
            #tokens.append(wp[0])
            #ner_tags.append(wp[1])
        
        
        if "I#-#H#U#M#A#N" in w:
            wp = w.replace("___I#-#H#U#M#A#N", "")
            wp = wp.replace("___I#-#S#P#E#C#I#E", "") #To clean multiple tags
            tokens.append(wp)
            ner_tags.append("I-HUMAN")
        elif "I#-#S#P#E#C#I#E" in w:
            wp = w.replace("___I#-#S#P#E#C#I#E", "")
            wp = wp.replace("___I#-#H#U#M#A#N", "")
            tokens.append(wp)
            ner_tags.append("I-SPECIE")
        else:
            tokens.append(w)
            ner_tags.append("O")
        if w == ".":
            dicc = {"ner_tags": ner_tags, "tokens": tokens}
            f_out.write(str(dicc)+',\n')
            ner_tags = []
            tokens = []
    #print(tokens, len(tokens))
    #print(ner_tags, len(ner_tags))
f_out.write("]}")
f_out.close()

<IPython.core.display.Javascript object>